In [0]:
dbutils.widgets.dropdown(
    "layer",
    "bronze",
    ["bronze", "silver", "gold"]
)


In [0]:
layer = dbutils.widgets.get("layer")
print(layer)

In [0]:
layer = dbutils.widgets.get("layer")

layer_paths = {
    "bronze": "workspace.ecommerce.bronze",
    "silver": "workspace.ecommerce.silver",
    "gold": "workspace.ecommerce.gold"
}

source_path = layer_paths[layer]

In [0]:
print(source_path)
     

In [0]:
df = spark.table(source_path)

In [0]:
display(df)

In [0]:

def run_layer(layer):
    source_path = layer_paths[layer]

    if layer == "bronze":
        raw = spark.read.format("json").load(source_path)
        raw.write.mode("overwrite").saveAsTable(
            "workspace.bronze.ecommerce_data"
        )

    elif layer == "silver":
        df = spark.table("workspace.bronze.ecommerce_data")
        df_clean = df.dropDuplicates()
        df_clean.write.mode("overwrite").saveAsTable(
            "workspace.silver.ecommerce_data"
        )

    elif layer == "gold":
        df = spark.table("workspace.silver.ecommerce_data")
        agg = df.groupBy("event_type").count()
        agg.write.mode("overwrite").saveAsTable(
            "workspace.gold.ecommerce_data"
        )